# Part 4: Using GPU acceleration with PyTorch

In [0]:
# Execute this code block to install dependencies when running on colab
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

try: 
    import torchbearer
except:
    !pip install torchbearer

     |████████████████████████████████| 143kB 2.8MB/s 


## Manual use of `.cuda()`

Now the magic of PyTorch comes in. So far, we've only been using the CPU to do computation. When we want to scale to a bigger problem, that won't be feasible for very long.
|
PyTorch makes it really easy to use the GPU for accelerating computation. Consider the following code that computes the element-wise product of two large matrices:

In [0]:
import torch

t1 = torch.randn(1000, 1000)
t2 = torch.randn(1000, 1000)
t3 = t1*t2
print(t3)

tensor([[ 0.5165,  2.5909,  0.1231,  ..., -0.5731,  0.5420,  1.2870],
        [ 1.9359, -0.5166, -0.2656,  ..., -0.0513,  0.4513,  0.6531],
        [-0.1323, -0.1499, -1.3790,  ...,  1.1154, -0.2466, -0.3105],
        ...,
        [-0.1526,  0.0347, -0.1880,  ...,  1.5437, -2.4183,  2.6595],
        [-0.1670,  0.2456,  1.6469,  ..., -0.8773, -0.1466, -1.1859],
        [-0.7291, -0.7334,  0.0378,  ..., -0.2863, -0.0523,  0.7893]])


By sending all the tensors that we are using to the GPU, all the operations on them will also run on the GPU without having to change anything else. If you're running a non-cuda enabled version of PyTorch the following will throw an error; if you have cuda available the following will create the input matrices, copy them to the GPU and perform the multiplication on the GPU itself:

In [0]:
t1 = torch.randn(1000, 1000).cuda()
t2 = torch.randn(1000, 1000).cuda()
t3 = t1*t2
print(t3)

tensor([[ 0.4778,  1.4390,  0.9644,  ..., -0.0225, -0.0637,  1.2161],
        [ 1.1111, -0.2887,  1.9724,  ..., -0.0048,  1.1970, -0.2505],
        [ 2.1519, -1.2513, -0.4204,  ...,  0.0770, -0.4185,  0.3397],
        ...,
        [-0.6936, -0.1098,  0.1007,  ...,  0.5673, -0.1350,  1.0333],
        [-2.0574, -0.5563,  0.7930,  ..., -0.5967, -0.0696,  1.1552],
        [-0.0502, -1.0634,  0.4203,  ..., -0.3923,  0.0853, -0.1941]],
       device='cuda:0')


If you're running this workbook in colab, now enable GPU acceleration (`Runtime->Runtime Type` and add a `GPU` in the hardware accelerator pull-down). You'll then need to re-run all cells to this point.

If you were able to run the above with hardware acceleration, the print-out of the result tensor would show that it was an instance of `cuda.FloatTensor` type on the the `(GPU 0)` GPU device. If your wanted to copy the tensor back to the CPU, you would use the `.cpu()` method.

## Writing platform agnostic code

Most of the time you'd like to write code that is device agnostic; that is it will run on a GPU if one is available, and otherwise it would fall back to the CPU. The recommended way to do this is as follows:

In [0]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
t1 = torch.randn(1000, 1000).to(device)
t2 = torch.randn(1000, 1000).to(device)
t3 = t1*t2
print(t3)

tensor([[-1.9530e-01, -3.9137e-01, -9.9981e-02,  ...,  2.3099e+00,
         -6.7034e-02, -2.5403e-01],
        [ 1.7326e-01,  3.4993e-01, -3.6272e-01,  ...,  5.7981e-01,
          3.7893e+00, -3.8336e-01],
        [ 6.1642e-01,  5.1971e-02, -2.6109e-01,  ...,  1.8660e+00,
         -1.2631e-01,  8.4604e-01],
        ...,
        [-5.1657e-01,  1.8378e+00, -8.3549e-04,  ...,  1.3673e-01,
         -1.6528e+00,  4.9166e-02],
        [ 6.6678e-01, -1.9724e-01,  1.2457e-01,  ...,  4.9920e-02,
         -1.1971e-02, -6.2089e-03],
        [ 3.3767e-02,  1.9892e+00, -1.0372e+00,  ..., -4.4273e-01,
         -6.7183e-01,  6.1327e-02]], device='cuda:0')


## Accelerating neural net training

If you wanted to accelerate the training of a neural net using raw PyTorch, you would have to copy both the model and the training data to the GPU. Unless you were using a really small dataset like MNIST, you would typically _stream_ the batches of training data to the GPU as you used them in the training loop:

```python
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = BaselineModel(784, 784, 10).to(device)

loss_function = ...
optimiser = ...

for epoch in range(10):
    for data in trainloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimiser.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimiser.step()
```

Using Torchbearer, this becomes much simpler - you just tell the `Trial` to run on the GPU and that's it!:

```python
model = BetterCNN()

loss_function = ...
optimiser = ...

device = "cuda:0" if torch.cuda.is_available() else "cpu"
trial = Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy']).to(device)
trial.with_generators(trainloader)
trial.run(epochs=10)
```


## Multiple GPUs

Using multiple GPUs is beyond the scope of the lab, but if you have multiple cuda devices, they can be referred to by index: `cuda:0`, `cuda:1`, `cuda:2`, etc. You have to be careful not to mix operations on different devices, and would need how to carefully orchestrate moving of data between the devices (which can really slow down your code to the point at which using the CPU would actually be faster).

## Questions

__Answer the following questions (enter the answer in the box below each one):__

__1.__ What features of GPUs allow them to perform computations faster than a typically CPU?

YOUR ANSWER HERE<br>
https://www.quora.com/Why-does-a-GPU-perform-certain-calculations-faster-than-a-CPU<br>
https://www.zhihu.com/question/19717029

__2.__ What is the biggest limiting factor for training large models with current generation GPUs?

YOUR ANSWER HERE<br>
https://medium.com/syncedreview/how-to-train-a-very-large-and-deep-model-on-one-gpu-7b7edfe2d072